[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [1]:
# Import pandas, numpy
import pandas as pd
import numpy as np
import seaborn as sns

In [6]:
# Read dataset with extra params sep='\t', encoding="latin-1"
lt_data = pd.read_csv('./labeledTrainData.tsv', sep='\t', encoding='latin-1')
lt_data.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


## 2. Preprocessing

In [7]:
# stop words
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/macos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from nltk.corpus import stopwords
stp_wrds = stopwords.words('english')

In [16]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    # Your code here
    text = re.sub('<[^>]*>','',text)
    # Save emoticons for later appending
    # Your code here
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    return text

lt_data['Processed_Reviews'] = lt_data.review.apply(lambda x: preprocessor(x))

In [17]:
lt_data.head()

,id,sentiment,review,Processed_Reviews
0,5814_8,1,With all this stuff going down at the moment w...,with all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",the classic war of the worlds by timothy hine...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,the film starts with a manager nicholas bell g...
3,3630_4,0,It must be assumed that those who praised this...,it must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious 8...


In [13]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()

def tokenizer_porter(text):
    token = [porter.stem(word) for word in text.split()]
    return token# Your code here


In [14]:
# split the dataset in train and test
# Your code here

from sklearn.model_selection import train_test_split
X = lt_data['Processed_Reviews']
y = lt_data['sentiment']
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size = 0.3)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [20]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(stop_words=stp_wrds,tokenizer=tokenizer_porter)# Your code here

clf = Pipeline([('vect',tfidf),('clf',LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

/Users/macos/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/Users/macos/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...e, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

## 4. Evaluate Model

In [22]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
prediction = clf.predict(X_test)
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))

0.8865333333333333
[[3285  504]
 [ 347 3364]]
              precision    recall  f1-score   support

           0       0.90      0.87      0.89      3789
           1       0.87      0.91      0.89      3711

   micro avg       0.89      0.89      0.89      7500
   macro avg       0.89      0.89      0.89      7500
weighted avg       0.89      0.89      0.89      7500



In [27]:
twits = [
    "This is really bad",
    "I love this!",
    "What the ........... is this ? This must, without a doubt, be the biggest waste of film, settings and camera ever. I know you can't set your expectations for an 80's slasher high, but this is too stupid to be true. I baught this film for 0.89$ and I still feel the urge to go claim my money back. Can you imagine who hard it STINKS ?<br /><br />Who is the violent killer in this film and what are his motivations??? Well actually, you couldn't possible care less. And why should you? The makers of this piece of garbage sure didn't care. They didn't try to create a tiny bit of tension. The director ( Stephen Carpenter -- I guess it's much easier to find money with a name like that ) also made the Kindred (1986) wich was rather enjoyable and recently he did Soul Survivors. Complete crap as well, but at least that one had Eliza Dushku. This junk has the debut of Daphne Zuniga !!! ( Who ?? ) Yeah that's right, the Melrose Place chick. Her very memorable character dies about 15 min. after the opening credits. She's the second person to die. The first victim dies directly in the first minute, but nobody seems to mention or miss him afterwards so who cares ? The rest of the actors...they don't deserve the term actors actually, are completely uninteresting. You're hoping they die a quick and painful death...and not only their characters<br /><br />My humble opinion = 0 / 10",
]

preds = clf.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> Negative, Positive = {preds[i]}')

This is really bad --> Negative, Positive = [0.99568546 0.00431454]
I love this! --> Negative, Positive = [0.00890148 0.99109852]
What the ........... is this ? This must, without a doubt, be the biggest waste of film, settings and camera ever. I know you can't set your expectations for an 80's slasher high, but this is too stupid to be true. I baught this film for 0.89$ and I still feel the urge to go claim my money back. Can you imagine who hard it STINKS ?<br /><br />Who is the violent killer in this film and what are his motivations??? Well actually, you couldn't possible care less. And why should you? The makers of this piece of garbage sure didn't care. They didn't try to create a tiny bit of tension. The director ( Stephen Carpenter -- I guess it's much easier to find money with a name like that ) also made the Kindred (1986) wich was rather enjoyable and recently he did Soul Survivors. Complete crap as well, but at least that one had Eliza Dushku. This junk has the debut of Dap

## 5. Export Model 

In [25]:
# Using pickle to export our trained model
import pickle
import os
pickle.dump(clf, open(os.path.join('data', 'logisticRegression.pkl'), 'wb'), protocol=4)